# Exploration in Linear Bandits

The objective of this part is to implement and compare the following strategies for linear bandits:

[Optimism in the Face of Uncertainty (LinUCB/OFUL)](https://papers.nips.cc/paper/4417-improved-algorithms-for-linear-stochastic-bandits.pdf)

[Thompson Sampling](https://projecteuclid.org/euclid.ejs/1513306870)

## Linear bandit
We consider the standard linear bandit setting. At each time $t$, the agent selects an arm $a_t \in A$ and observes a reward
$$
r_{a}^t = \langle \theta^\star, \phi_a^t \rangle + \eta_a^t := \mu_a^t + \eta_a^t
$$
where $\theta^{\star} \in \mathbb{R}^{d}$ is a parameter vector, $\phi_{a}^t \in \mathbb{R}^{d} $ are the features of arm $a$ at time $t$, and $\eta_{a}^{t}$ is a zero-mean  $\sigma^2$-subgaussian noise. 

When the features correspond to the canonical basis, this formulation reduces to multi-armed bandit (MAB) with $d$ arms. In the more general case, the features may depend on a context $x_t$, so that $\phi_a^t = \phi(x_t, a)$ denotes the feature vector of a context-action pair $(x_t, a)$ and the resulting setting is the so-called linear contextual bandit.

We rely on the following standard assumption on the features and the unknown parameter $\theta^\star$.

**Assumption.** There exist $B,D \geq 0$, such that $\|\theta^\star\|_2 \leq B$, $\|\phi_a^t\| \leq D$, and $\langle \theta^\star, \phi_a^t \rangle \in [0,1]$, for all $t$ and $a$.

Given a finite horizon $n$, the performance of the agent is measured by its (pseudo)-*regret*:
$$
        %R(n) = n \mu^{\star} - \sum_{t=1}^n \mu_{a_t} = \sum_{i=1}^K T_i(n) \Delta_i, 
        R(n) = \sum_{t=1}^n \langle \theta^\star, \phi_{a^\star}^t \rangle - \langle \theta^\star, \phi_{a_t}^t \rangle ,
$$
where $a^{\star}_{t} \in \arg\max_{a} \langle \theta^\star, \phi_{a}^t \rangle$ is the optimal action at time $t$.

**We consider the simple linear bandit setting:** $\phi_a^t = \phi_a, \; \forall t$




In [1]:
!rm -rf mvarl_hands_on/
!git clone https://github.com/rlgammazero/mvarl_hands_on.git
!cd mvarl_hands_on/ && git fetch
!ls mvarl_hands_on/utils

Cloning into 'mvarl_hands_on'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 39 (delta 14), reused 37 (delta 12), pack-reused 0
Unpacking objects: 100% (39/39), done.
cliffwalk.py  discretization.py  frozen_lake.py  test_env.py
coldstart.py  finite_env.py	 gridworld.py


In [2]:
import sys
sys.path.insert(0, './mvarl_hands_on/utils')
import os
import numpy as np
from pprint import pprint
from coldstart import ColdStartFromDataset
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import math
from tqdm import tqdm

#### Jester Jokes Dataset (Dense subset of 40 jokes)

Deep Bayesian Bandits Showdown: An Empirical Comparison of Bayesian Deep Networks for Thompson Sampling

Download the data at: https://storage.googleapis.com/bandits_datasets/jester_data_40jokes_19181users.npy

We performed a matrix factorization of the ratings (after filtering over users and jokes). This provides features for the arms and users, the reward (ie rating) is the dot product between the arm and user features (we make it stochastic by adding Gaussian noise). We consider a cold start problem where the user is randomly selected at the beginning of the repetition and the agent has to learn the best arm to recommend. When an arm is selected by the algorithm, its reward is computed as the dot product between the arm and user features.

In [3]:
!wget https://storage.googleapis.com/bandits_datasets/jester_data_40jokes_19181users.npy

--2019-12-30 00:14:35--  https://storage.googleapis.com/bandits_datasets/jester_data_40jokes_19181users.npy
Resolving storage.googleapis.com (storage.googleapis.com)... 2a00:1450:4007:815::2010, 216.58.204.144
Connecting to storage.googleapis.com (storage.googleapis.com)|2a00:1450:4007:815::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6138000 (5.9M) [application/octet-stream]
Saving to: ‘jester_data_40jokes_19181users.npy’

jester_data_40jokes 100%[===================>]   5.85M  10.1MB/s    in 0.6s    

2019-12-30 00:14:36 (10.1 MB/s) - ‘jester_data_40jokes_19181users.npy’ saved [6138000/6138000]



In [4]:
M = np.load('jester_data_40jokes_19181users.npy')
M = M / 10
K = 35
U, s, Vt = svds(M, k = K)
s=np.diag(s)
U = np.dot(U,s)
print('U: {}'.format(U.shape))
print('Vt: {}'.format(Vt.shape))
print('#features: {}'.format(Vt.shape[0]))
print('#arms: {}'.format(Vt.shape[1]))
np.savetxt('U_jester.csv', U, delimiter=',') 
np.savetxt('Vt_jester.csv', Vt, delimiter=',') 

U: (19181, 35)
Vt: (35, 40)
#features: 35
#arms: 40


Create the coldstart model

In [5]:
seed = 1235
user_subset = np.linspace(0, 400, 10).astype(int).tolist()
arm_csvfile = os.path.abspath('Vt_jester.csv')
user_csvfile = os.path.abspath('U_jester.csv')
noise_std = 0.1

config_cs = {
    'arm_csvfile': arm_csvfile,
    'user_csvfile': user_csvfile,
    'random_state': seed,
    'user_subset': user_subset,
    'noise_std': noise_std
}

print("Current config is:")
pprint(config_cs)


Current config is:
{'arm_csvfile': '/home/naunauyoh/PycharmProjects/MVA_RL/Vt_jester.csv',
 'noise_std': 0.1,
 'random_state': 1235,
 'user_csvfile': '/home/naunauyoh/PycharmProjects/MVA_RL/U_jester.csv',
 'user_subset': [0, 44, 88, 133, 177, 222, 266, 311, 355, 400]}


In [6]:
noise = 0.2
random_state = 312
model = ColdStartFromDataset(**config_cs)
print("\nThe new user arriving to the system is user #", model.theta_idx)
print("\nTheta*: ", model.theta)
means = np.dot(model.features, model.theta)
print("\nMeans: ", means)
theta_bound = np.linalg.norm(model.theta, 2)
print("\nTheta bound: ", theta_bound)


The new user arriving to the system is user # 133

Theta*:  [ 0.07360712  0.17734693  0.20495631 -0.22209593  0.19274253 -0.18410772
  0.11715996 -0.46113109 -0.07918465  0.06932272  0.08971403 -0.17425277
  0.22851053  0.16756964 -0.9522754   0.33699145  0.3933289  -0.74392791
 -0.26068602  0.63669768 -0.24925958  0.84903929 -0.04273235 -0.62347351
 -0.1563364  -0.54413666  0.48877408 -0.45984457 -0.86347719  0.21857714
 -0.36691055 -0.73139386 -1.43376962 -0.03908683 -1.71844609]

Means:  [0.80575473 0.1709222  0.40801812 0.67453197 0.03911792 0.07580014
 0.74331481 0.39452305 0.58721524 1.         0.96657887 0.65363789
 0.48448818 0.13126515 0.73118925 0.78475723 0.84698373 0.73243179
 0.69016416 0.20050761 0.91706563 0.53946859 0.67640884 0.05161643
 0.76192058 0.51896502 0.04415654 0.         0.76867243 0.33779268
 0.08463928 0.05049376 0.49854492 0.69503763 0.12060601 0.14961957
 0.08393364 0.55350756 0.67109645 0.85149357]

Theta bound:  3.3428890242516553


**Question 1**: implement LinUCB

### LinUCB / OFUL
See the slides!

Note that it is not necessary to invert the matrix $A_t$ at each round. Since $A_t$ is obtained from a rank-1 update of $A_{t-1}$, it is possible to use Sherman–Morrison formula to build directly $A_t^{-1}$.

Suppose $𝐴$ be a nonsingular $n\times n$ matrix and $\mathbf{u}, \mathbf{v}$ be vectors. Then
$$
(A+\mathbf{u}\mathbf{v}^T)^{-1} = A^{-1} - \frac{A^{-1}\mathbf{u}\mathbf{v}^TA^{-1}}{1+\mathbf{v}^TA^{-1}\mathbf{u}}.
$$

In [0]:
class OFUL:
    def __init__(self, arm_features, reg_factor, delta,
                 bound_theta, noise_std):
        
        self.arm_features = arm_features
        self.reg_factor = reg_factor
        self.delta = delta
        self.iteration = 0
        self.bound_theta = bound_theta
        self.bound_features = np.max(np.sqrt(np.sum(np.abs(arm_features) ** 2, axis=1)))
        self.noise_std = noise_std
        
        self.A = np.eye(self.n_features)
        self.A_inv = np.eye(self.n_features)
        self.b = np.zeros((self.n_features, 1))
        self.theta = np.zeros((self.n_features, 1))

        self.reset()

    @property
    def n_actions(self):
        return self.arm_features.shape[0]

    @property
    def n_features(self):
        return self.arm_features.shape[1]
    
    def reset(self):
        """Return the internal estimates
        Not sure about the naming of this function ... 
        """
        estimates = []
        for arm_feature in arm_features:
            estimates.append(arm_feature.dot(self.theta))
        
        return np.array(estimates)
        
    def alpha(self, n_samples):
        # TODO remove the n_samples (?) as it seems equivalent to self.iteration
        return self.noise_std * np.sqrt(self.n_features * 
                                        np.log((1 + self.iteration * self.bound_features / self.reg_factor) / self.delta)) + \
                                np.sqrt(self.reg_factor) * self.bound_theta
                        
        
    def sample_action(self):
        """Return the action to play based on current estimates
        """
        action = None
        val = None
        alpha = self.alpha()
        estimates = self.reset()
        for i, estimate in enumerate(estimates):
            B = estimate + alpha * np.sqrt(self.arm_features[i].dot(self.A_inv).dot(self.arm_features[i]))
            if action is None or val < B:
                action = i
                val = B

        return action
    
    def update(self, a_t, r_t):
        """Update the estimates of the model
        

        Parameters
        ----------
        a_t: int
            The action played at the current episode
        r_t: float
            The reward associated to action a_t

        Returns
        -------
        none
        """
        features = self.arm_features[a_t]
        self.A += np.outer(features,features)
        
        self.b += r_t*features
        
        self.A_inv = self.A_inv - self.A_inv.dot(np.outer(features,features)).dot(self.A_inv) / \
            (1 + features.dot(self.A_inv).dot(features))
        
        self.theta = self.A_inv.dot(self.b)

        

**Question 2:** implement LinearTS

Let $A_t$ be the design matrix, $\theta_t$ the estimate of $\theta^\star$ and $\beta_t$ the confidence interval built by LinUCB. Then, at every time step t,  LinearTS simply generates
a sample $\tilde{\theta}_t$ from the distribution $\mathcal{N}(\widehat{\theta}_t, \omega_t \alpha_t^2 A_t^{-1})$.

LinearTS

For $t=1, \ldots, T$
> $\tilde{\theta}_t \sim \mathcal{N}(\widehat{\theta}_t, \omega_t \alpha_t^2 A_t^{-1})$
>
> $a_t \in \arg\max_{a \in \mathcal{A}_t}  \langle \tilde{\theta}_t, \phi_{a} \rangle$
>
> observe reward $r_t$

TS is requires to draw $\tilde{\theta}_t$ from a distribution over-sampling by a factor $\sqrt{d}$ the ellipsoid constructed by OFUL (i.e., $\omega_t = d$). This is required to prove that LinearTS is optimistic with a fix probability. This is necessary to prove the frequentist regret of TS. The regret of TS is worse than the one of LinUCB by a factor $\sqrt{d}$ (i.e., $\widetilde{O}(d^{3/2}\sqrt{T})$)

In [0]:
class LinearTS(OFUL):
    def __init__(self, arm_features, reg_factor, delta,
                 bound_theta, noise_std):
        super(LinearTS, self).__init__(arm_features, reg_factor, delta,
                 bound_theta, noise_std)
        self.alpha = 
        
    #TODO

**Question 3**: run the algorithms (`LinUCB` and `LinearTS`) and average the performance over multiple users (ie simulations)

The regret $R(T) = \sum_t \phi_t^\top (\theta^\star - \theta_t)$

The performance is the expected regret over multiple users. You can also test `LinearTS` without the additional $\sqrt{d}$.

You can use `RandomLinearArms` to test your code before using Jester

In [0]:
# from mvarl_hands_on.utils.coldstart import RandomLinearArms

nb_simulations = 4
T = int(4e4)

algorithms = {
            'OFUL': lambda arm_features, bound_theta: 
              OFUL(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std']),
             'LinearTS': lambda arm_features, bound_theta:
              LinearTS(arm_features=arm_features, reg_factor=1., delta=0.01,
                 bound_theta=bound_theta, noise_std=config_cs['noise_std'])
             }

regrets = {}

for alg_name in algorithms.keys():
    if alg_name not in regrets.keys():
        regrets[alg_name] = np.zeros((nb_simulations, T))
    
    for k in range(nb_simulations):
        if k % 1 == 0:
            print("{} simulation {}/{}".format(alg_name, k+1, nb_simulations))
        model = ColdStartFromDataset(**config_cs)
#         model = RandomLinearArms()
        alg = algorithms[alg_name](arm_features=, bound_theta=)

        #TODO: implement interaction loop
        #for t
        #   ....
        #   regrets[alg_name][k, t] = 


In [0]:
plt.figure(figsize=(10,8))
for alg_name in regrets.keys():
    data = np.cumsum(regrets[alg_name], axis=1)
    n_rep, T = data.shape
    
    mean_regret = np.mean(data, axis=0)
    std_regret = np.std(data, axis=0) / math.sqrt(n_rep)
    t = np.arange(T)
    plt.plot(t, mean_regret, label=alg_name)
    plt.fill_between(t, mean_regret - 2 * std_regret, mean_regret + 2 * std_regret, alpha=0.15)
plt.legend()
    